# 109: Swissmetro Nested Logit Mode Choice

In [ ]:
# TEST

import numpy as np
import pandas as pd

import larch

pd.set_option("display.max_columns", 999)
pd.set_option("expand_frame_repr", False)
pd.set_option("display.precision", 3)
np.set_printoptions(precision=12)
larch._doctest_mode_ = True
import larch as lx

In [ ]:
import pandas as pd

import larch as lx

This example is a mode choice model built using the Swissmetro example dataset.
First we create the Dataset and Model objects. 

In [ ]:
raw_data = pd.read_csv(lx.example_file("swissmetro.csv.gz")).rename_axis(index="CASEID")
data = lx.Dataset.construct.from_idco(raw_data, alts={1: "Train", 2: "SM", 3: "Car"})
m = lx.Model(data.dc.query_cases("PURPOSE in (1,3) and CHOICE != 0"))

We can attach a title to the model. The title does not affect the calculations
as all; it is merely used in various output report styles.

In [ ]:
m.title = "swissmetro example 09 (nested logit)"

We need to identify the availability and choice variables.

In [ ]:
m.availability_co_vars = {
    1: "TRAIN_AV * (SP!=0)",
    2: "SM_AV",
    3: "CAR_AV * (SP!=0)",
}
m.choice_co_code = "CHOICE"

The swissmetro dataset, as with all Biogeme data, is only in `co` format.

In [ ]:
from larch import P, X

m.utility_co[1] = P.ASC_TRAIN + P.B_TIME * X.TRAIN_TT + P.B_COST * X("TRAIN_CO*(GA==0)")
m.utility_co[2] = P.B_TIME * X.SM_TT + P.B_COST * X("SM_CO*(GA==0)")
m.utility_co[3] = P.ASC_CAR + P.B_TIME * X.CAR_TT + P.B_COST * X("CAR_CO")

To create a new nest, we can use the `graph.new_node` command.
For this example, we want to nest together the Train and Car modes into a "existing" modes nest.
Those are modes 1 and 3, so we can use the `graph.new_node` command like this:

In [ ]:
m.graph.new_node(parameter="existing", children=[1, 3], name="Existing")

In a Jupyter notebook, we can verify the nesting structure visually if we like.

In [ ]:
m.graph

Larch will find all the parameters in the model, but we'd like to output them in
a rational order.  We can use the ordering method to do this:

In [ ]:
m.ordering = [
    (
        "ASCs",
        "ASC.*",
    ),
    (
        "LOS",
        "B_.*",
    ),
    ("LogSums", "existing"),
]

In [ ]:
# TEST
from pytest import approx

assert m.loglike() == approx(-6964.662979192185)

We can estimate the models and check the results match up with those given by Biogeme:

In [ ]:
m.set_cap(15)
result = m.maximize_loglike()
result

In [ ]:
# TEST
assert result.loglike == approx(-5236.900013608126)
assert result.logloss == approx(0.7737736426725954)
assert result.x == approx(
    [
        -0.167120809449,
        -0.512054631478,
        -0.008569762572,
        -0.008988398761,
        0.486972985782,
    ],
    rel=1e-2,
)

In [ ]:
m.calculate_parameter_covariance(robust=True)
m.parameter_summary()

In [ ]:
# TEST
assert m.pstderr == approx(
    [0.037138242, 0.045185346, 0.00046280216, 0.00056991476, 0.027901005], rel=1e-2
)
assert m.parameter_summary().data["t Stat"].values.astype(float) == approx(
    [-4.5, -11.33, -18.52, -15.77, -18.39], rel=1e-2
)
assert m.parameter_summary().data["Signif"].values == approx(
    ["***", "***", "***", "***", "***"]
)
assert m.parameter_summary().data["Robust t Stat"].values.astype(float) == approx(
    [-3.06, -6.47, -14.27, -8.39, -13.18], rel=1e-2
)
assert m.parameter_summary().data["Robust Signif"].values == approx(
    ["**", "***", "***", "***", "***"]
)